### Module -1: Analyzing Improperly parsed rationales

In [1]:
incorrect_eval = '755fdd7c08a411ebbd7cac1f6bf848b6'

response1 ={"response": "First, the director of Broadway Or Bust was born on 28 June 1915. Second, the director of Ballad Of Tara was born on 27 April 1936. The director of Broadway Or Bust was born first.", "id": "755fdd7c08a411ebbd7cac1f6bf848b6", "question": "Which film whose director was born first, Broadway Or Bust or Ballad Of Tara?", "consistency": 0.8, "rationale": "none", "answer": "NOT ENOUGH INFO", "original_answers": ["Broadway Or Bust", "NOT ENOUGH INFO", "NOT ENOUGH INFO", "NOT ENOUGH INFO", "NOT ENOUGH INFO"], "original_rationales": ["First, the director of Broadway Or Bust is William Beaudine, who was born on January 15, 1892. Second, the director of Ballad Of Tara is R. Nataraja Mudaliar, who was born on October 31, 1896.", "none", "none", "none", "none"], "high_consistency": True, "time_taken_sec": 1.43}

In [3]:

response2 = {"response": "First, Charles Of Provence's paternal grandfather is Charles Martel.", "id": "9ec997120baf11ebab90acde48001122", "question": "Who is Charles Of Provence's paternal grandfather?", "consistency": 0.4, "rationale": "none", "answer": "NOT ENOUGH INFO", "original_answers": ["Charles III of France", "Louis The Pious", "NOT ENOUGH INFO", "Louis the Stammerer", "NOT ENOUGH INFO"], "original_rationales": ["First, Charles Of Provence's paternal grandfather is Charles III of France. Second, Charles III of France was also known as Charles the Simple.", "First, Charles Of Provence's paternal grandfather is Louis The German. Second, Louis The German was the son of Louis The Pious.", "none", "First, Charles Of Provence's paternal grandfather is Louis the Stammerer. Second, Louis the Stammerer was the King of Western Francia.", "none"], "consistency_time": 0.95, "high_consistency": False, "verifying_questions": ["Who is Charles Of Provence's father?"], "time_verifying_answer": 0.64, "time_context_and_verifying_answer": 10.11, "context": [["Charles was only a child when his father died; and the governance of his realm was undertaken by his tutor, Count Girart de Roussillon, whose wife had been a sister-in-law of Lothar I. Girart was a vigorous regent, defending the kingdom from the Northmen, who raided up the Rhone as far as Valence.", "Charles II, also known as Charles the Lame (French: Charles le Boiteux; Italian: Carlo lo Zoppo; 1254 \u2013 5 May 1309), was King of Naples, Count of Provence and Forcalquier (1285\u20131309), Prince of Achaea (1285\u20131289), and Count of Anjou and Maine (1285\u20131290); he also was King of Albania (1285\u20131294), and claimed the Kingdom of Jerusalem from 1285.", "The series, created by Sydney Newman, C. E. Webber and Donald Wilson, depicts the adventures of an extraterrestrial being called the Doctor, part of a humanoid species called Time Lords."]], "verifying_answers": ["Charles Of Provence's father was Charles II."], "time_answer_again": 0.76, "new_answer": " Second, Charles Of Provence's paternal grandfather was Louis the Stammerer. The answer is Louis the Stammerer.", "time_taken_sec": 12.46}
response2

{'response': "First, Charles Of Provence's paternal grandfather is Charles Martel.",
 'id': '9ec997120baf11ebab90acde48001122',
 'question': "Who is Charles Of Provence's paternal grandfather?",
 'consistency': 0.4,
 'rationale': 'none',
 'answer': 'NOT ENOUGH INFO',
 'original_answers': ['Charles III of France',
  'Louis The Pious',
  'NOT ENOUGH INFO',
  'Louis the Stammerer',
  'NOT ENOUGH INFO'],
 'original_rationales': ["First, Charles Of Provence's paternal grandfather is Charles III of France. Second, Charles III of France was also known as Charles the Simple.",
  "First, Charles Of Provence's paternal grandfather is Louis The German. Second, Louis The German was the son of Louis The Pious.",
  'none',
  "First, Charles Of Provence's paternal grandfather is Louis the Stammerer. Second, Louis the Stammerer was the King of Western Francia.",
  'none'],
 'consistency_time': 0.95,
 'high_consistency': False,
 'verifying_questions': ["Who is Charles Of Provence's father?"],
 'time_ve

### Module 0: Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import pprint
import os 
from time import time 
from dotenv import load_dotenv
import json
import argparse
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from openai import OpenAI
import certifi

# Override bad SSL_CERT_FILE if set
os.environ["SSL_CERT_FILE"] = certifi.where()

from sentence_transformers import SentenceTransformer
import torch
import sklearn

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
## Importing VE libraries
from prompt_cot import cot_prompt
from utils import *
from dataset_utils import read_wikiqa_data
from prompt_helper import get_joint_prompt_helper, normalize_prediction
from dataset_utils import read_wikiqa_data, wiki_evaluation


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
load_dotenv()
client = OpenAI()

In [5]:
from consistency import _parse_args as consistency_parse_args, in_context_manual_prediction  as consistency_in_context_manual_prediction, post_process_consistency, evaluate_manual_predictions as consistency_evaluate_manual_predictions
from verifying_questions import _parse_args as verifying_questions_args, in_context_manual_prediction as vq_in_context_manual_prediction
from relevant_context import _parse_args as relevant_context_args
from verifying_answers import in_context_manual_prediction as verifying_answers_in_context_manual_prediction, _parse_args as verifying_answers_args
from answer_again import rationale as answer_again_rationale, in_context_manual_prediction as answer_again_in_context_manual_prediction
from evaluation import evaluate_model_performance_v2

### Module 1: Building Consolidated function

In [6]:
# model embedding function required for relevant context
def model_embeddings(sentence, model):
    embedding = model.encode([sentence])
    return embedding[0] #should return an array of shape 384

In [7]:
def run_iterative_prediction():

    # Consistency Arguments
    args = consistency_parse_args()

    output_file = f"misc/iterative_predictions_cot_final_iter_new_0_250.jsonl"

    # Load existing IDs from output file
    existing_ids = set()
    if os.path.exists(output_file):
        with open(output_file, "r") as f:
            for line in f:
                try:
                    data = json.loads(line.strip())
                    existing_ids.add(data["id"])
                except:
                    continue

    # train_set = read_wikiqa_data("data/train_subset.json", manual_annotation_style=args.style)
    # train_set = train_set[args.train_slice:(args.train_slice + args.num_shot)]

    dev_set = read_wikiqa_data("data/dev_sampled.json")
    dev_set = dev_set[args.dev_slice:(args.dev_slice + args.num_dev)]

    results = []
    total_start = time()

    for idx, ex in enumerate(tqdm(dev_set, desc="Processing one-by-one")):
        if ex['id'] in existing_ids:
            continue

        start = time()
        try:
            # Step 1: Predict
            raw_pred = consistency_in_context_manual_prediction(ex, cot_prompt, engine=args.engine, prompt_helper=args.helper, length_test_only=False, n=args.num_shot)
            
            # Step 2: Consistency post-processing
            con_score, final_pred = post_process_consistency(ex, raw_pred, args)
            final_pred["consistency"]= con_score

            # Step  3: check if the consistency is higher than the threshold
            args =verifying_questions_args()

            if con_score > args.consistency_threshold:
                # Step 4: Storing the result if consistency is higher than the threshold 
                final_pred['high_consistency'] = True
                final_pred["time_taken_sec"] = round(time() - start, 2)
                with open(output_file, "a") as fout:
                    fout.write(json.dumps(final_pred) + "\n")

            else:
                start2 = time()
                final_pred["consistency_time"] = round(start2 - start, 2)
                final_pred['high_consistency'] = False
                # Step 5: Generating verifying questions
                vq = []
                sentences = rationale_tokenize(final_pred['rationale'])
                for q, s in enumerate(sentences):
                    question = vq_in_context_manual_prediction(final_pred['question'], s, args.engine, args.model, args.helper)
                    if question != None:
                        vq.append(question['text'])
                
                final_pred["verifying_questions"] = vq

                # Step 5: Pulling Relevant Context
                start3 = time()
                final_pred["time_verifying_answer"] = round(start3 - start2, 2)
                args = relevant_context_args()

                contexts = []
                embedding_model = SentenceTransformer('paraphrase-MiniLM-L6-v2').to(device)

                sentences = rationale_tokenize(final_pred['rationale'])
                all_pars_text = []
                all_pars = []
                va = []
                for j, s in enumerate(sentences):
                    pars_text = get_texts_to_rationale_wikipedia(vq[j], False)
                
                    pars_text = list(dict.fromkeys(pars_text)) #remove potential duplicates
                    all_pars_text += pars_text

                    if pars_text != []:
                            sen_embeds = [model_embeddings(s, embedding_model)]
                            par_embeds = [model_embeddings(s, embedding_model) for s in pars_text]

                            pars = sklearn.metrics.pairwise.pairwise_distances(sen_embeds, par_embeds)
                            pars = pars.argsort(axis = 1)[0][:args.topk]

                            pars = [pars_text[i] for i in pars]
                            contexts.append(pars)
                            all_pars += pars
                    
                    # Step 6: Creating Verified Answers
                    answer = verifying_answers_in_context_manual_prediction(args.engine, args.helper, args.model, pars, vq[j])
                    va.append(answer['text'].lstrip())
                
                start4 = time()
                final_pred["time_context_and_verifying_answer"] = round(start4- start3,2)
                final_pred["context"] = contexts
                final_pred["verifying_answers"] = va

                # Step 7: Answering Again
                new_rationale = answer_again_rationale(va)
                new_p = answer_again_in_context_manual_prediction(ex, cot_prompt, args.engine, args.helper, args.model,
                    new_rationale )

                start5 =time()
                final_pred["time_answer_again"] = round(start5 - start4, 2)
                final_pred["new_answer"] = new_p["text"]
                
                final_pred["time_taken_sec"] = round(time() - start, 2)
                with open(output_file, "a") as fout:
                    fout.write(json.dumps(final_pred) + "\n")
            
            results.append(final_pred)

            # # Step 4: Save this prediction to file immediately
            # with open(output_file, "a") as fout:
            #     fout.write(json.dumps(final_pred) + "\n")

        except Exception as e: 
            print(f"Error in record {idx} ({ex['id']}): {e}")
            continue

    total_duration = round(time() - total_start, 2)
    avg_time = total_duration / len(results) if results else 0
    print(f"\n✅ Finished {len(results)} records in {total_duration:.2f} seconds (avg {avg_time:.2f} sec/record)")
    print(f"📄 Results saved to: {output_file}")

    return None

In [9]:
run_iterative_prediction()

7 not found


Processing one-by-one:  27%|██▋       | 68/250 [00:14<00:52,  3.44it/s]

Error in record 67 (3314f9f708c311ebbd8dac1f6bf848b6): ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


Processing one-by-one: 100%|██████████| 250/250 [37:14<00:00,  8.94s/it]


✅ Finished 184 records in 2234.99 seconds (avg 12.15 sec/record)
📄 Results saved to: misc/iterative_predictions_cot_final_iter_new_0_250.jsonl


In [10]:
path = 'misc/iterative_predictions_cot_final_iter_new_0_250.jsonl'

In [11]:
df, result = evaluate_model_performance_v2(path)

In [12]:
result

{'exact_match_percentage': 16.87,
 'verify_percentage': 50.6,
 'average_time_per_record_sec': 11.81}

In [13]:
df.shape

(249, 20)

In [15]:
df.to_csv('misc/ve_predictions_cot_iter_latest_0_250.csv')

### Module 2: Analyzing Results

In [15]:
df.head()

,response,id,question,right_answer,consistency,rationale,answer,original_answers,original_rationales,consistency_time,high_consistency,verifying_questions,time_verifying_answer,time_context_and_verifying_answer,context,verifying_answers,time_answer_again,new_answer,time_taken_sec,match_flag
0,"First, the performer of the song ""Get A Life -...",1ee47f380bde11eba7f7acde48001122,Where was the performer of song Get A Life – G...,Vöcklabruck,0.2,"First, the performer of the song Get A Life - ...","New York, United States","[New York, United States, Detroit, Michigan, U...","[First, the performer of the song ""Get A Life ...",1.30,False,[What city was the performer of the song Get A...,1.22,17.84,[[Get a life or Get a Life may refer to:\n\nGe...,"[The performer of the song ""Get A Life - Get A...",1.78,The answer is Austria.,22.15,0
1,"First, the director of the film Don Juan (1922...",49eb87280bdc11eba7f7acde48001122,Where did the director of film Don Juan (1922 ...,Westerland,0.6,"First, the director of the film Don Juan (1922...","Hollywood, California","[Hollywood, California, United States, Hollywo...","[First, the director of the film Don Juan (192...",NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.43,0
2,"First, the composer of the film Naayudamma is ...",2e17069c0bde11eba7f7acde48001122,What is the place of birth of the composer of ...,Tamil,0.6,"First, Naayudamma is a Telugu film released in...","Pannaipuram, Tamil Nadu, India","[Pannaipuram, Tamil Nadu, India, Pannaipuram, ...","[First, the composer of the film Naayudamma is...",NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.12,0
3,"First, the director of the film Temptation (19...",28942dc20bdd11eba7f7acde48001122,Where did the director of film Temptation (195...,Nice,0.4,"First, the director of the film ""Temptation"" (...","Fårö, Sweden","[Fårö, Sweden, Hollywood, California, United S...","[First, the director of the film Temptation (1...",1.58,False,"[Where did Irving Pichel, the director of the ...",1.19,19.75,"[[Irving Pichel (June 24, 1891 – July 13, 1954...","[Irving Pichel, the director of the film ""Temp...",1.29,"The answer is Hollywood, California.",23.81,0
4,"First, John Kennedy, 2nd Lord Kennedy, has an ...",6a8a71280bb011ebab90acde48001122,"Who is the uncle of John Kennedy, 2Nd Lord Ken...",James Kennedy,0.2,"First, John Kennedy, 2nd Lord Kennedy, has an ...","Robert Kennedy, 1st Lord Kennedy","[Robert Kennedy, 1st Lord Kennedy, King James ...","[First, John Kennedy, 2nd Lord Kennedy, has an...",1.30,False,"[Who is the uncle of John Kennedy, 2nd Lord Ke...",1.47,22.03,[[Kennedy may refer to:\n\nPeople\nKennedy (su...,"[The uncle of John Kennedy, 2nd Lord Kennedy i...",0.60,The answer is Robert F. Kennedy.,25.41,0


In [17]:
df['high_consistency'].value_counts(normalize=True)

high_consistency
False    0.64
True     0.36
Name: proportion, dtype: float64

In [20]:
pd.set_option('display.max_colwidth',None)

In [23]:
df.loc[df.high_consistency ==False][['response']]

,response
0,"First, the performer of the song ""Get A Life - Get Alive"" is born in New York City. Second, New York City is located in the state of New York, United States. The answer is New York, United States."
3,"First, the director of the film Temptation (1959 Film) was Ingmar Bergman. Second, Ingmar Bergman died in Fårö, Sweden. The answer is Fårö, Sweden."
4,"First, John Kennedy, 2nd Lord Kennedy, has an uncle named Robert Kennedy, 1st Lord Kennedy. Second, Robert Kennedy, 1st Lord Kennedy, is the uncle of John Kennedy, 2nd Lord Kennedy. The answer is Robert Kennedy, 1st Lord Kennedy."
5,"First, the director of Broadway Or Bust is Tom Donahue, who was born on September 26, 1974. Second, the director of Ballad Of Tara is Peter Blystone, who was born on November 2, 1981. The answer is Tom Donahue, born on September 26, 1974."
8,"First, Charles Of Provence's father is Louis II of Italy. Second, Louis II of Italy's father is Lothair I. The answer is Lothair I."
9,"First, The Mate Of The Sally Ann was directed by Maurice Elvey, who was born on 11 November 1887. Second, The Legend Of Hercules was directed by Renny Harlin, who was born on 15 March 1959. The answer is Renny Harlin, who directed The Legend Of Hercules."
10,"First, Archduchess Maria Antonia Of Austria (1899–1977) was the daughter of Archduke Karl Ferdinand of Austria. Second, Archduke Karl Ferdinand of Austria's father was Archduke Karl Ludwig of Austria. The answer is Archduke Karl Ludwig of Austria."
11,"First, the director of ""Curry And Pepper"" is Gordon Chan, born on 23 December 1960. Second, the director of ""End Of Watch"" is David Ayer, born on January 18, 1968. The answer is Gordon Chan, born on 23 December 1960."
14,"First, the director of the film ""The Trap"" (1959 film) is Sidney Hayers. Second, Sidney Hayers passed away due to natural causes. The answer is natural causes."
16,"First, the director of the film Autumn Moon is Clara Law. Second, Clara Law is from Hong Kong. The answer is Hong Kong."


### Test Updated CoT

In [7]:

# Consistency Arguments
args = consistency_parse_args()


In [8]:
dev_set = read_wikiqa_data("data/dev_sampled.json")
# dev_set = dev_set[args.dev_slice:(args.dev_slice + args.num_dev)]

7 not found


In [16]:
id='755fdd7c08a411ebbd7cac1f6bf848b6'

In [17]:
test_dev =[i for i in dev_set if i["id"] == id][0]

In [18]:
test_dev

{'id': '755fdd7c08a411ebbd7cac1f6bf848b6',
 'question': 'Which film whose director was born first, Broadway Or Bust or Ballad Of Tara?',
 'answer': 'Broadway Or Bust',
 'all_pars': ['Bahrām Beyzāie (also spelt Beizai, Beyza\'i, , born 26 December 1938) is a critically and popularly acclaimed filmmaker, playwright, theatre director, screenwriter, film editor, and "ostādmaster") of Persian letters, arts and Iranian studies.',
  'Bahram Beyzaie is the son of the poet Ne\'matallah Beyzai (best known by his literary pseudonym "Zokā\'i").',
  "The celebrated poet Adib Beyzai, known as one of the most profound poets of 20th-century Iran, is Bahram's paternal uncle.",
  'Bahram Beyzaie\'s paternal grandfather, Mirzā Mohammad-Rezā Ārāni ("Ebn Ruh"), and paternal great-grandfather, the mulla Mohammad-Faqih Ārāni ("Ruh\'ol-Amin"), were also notable poets.',
  'In spite of his somewhat belated start in cinema, Beyzai is often considered a pioneer of a generation of filmmakers whose works are somet

In [13]:
test_dev["question"]

'Are the movies The Statement (Film) and A Self Made Hero, from the same country?'

In [19]:
raw_pred = consistency_in_context_manual_prediction(test_dev, cot_prompt, engine=args.engine, prompt_helper=args.helper, length_test_only=False, n=args.num_shot)

In [15]:
raw_pred

{'responses': ['First, The Statement (Film) is a 2003 film from France. Second, A Self Made Hero is a 1996 French film. The answer is France.',
  'First, The Statement (Film) is a movie from Canada. Second, A Self Made Hero is a movie from France. The answer is no.',
  'First, The Statement (Film) is a movie from France. Second, A Self Made Hero is also a movie from France. The answer is France.',
  'First, The Statement (Film) is a film from France. Second, A Self Made Hero is also a French film. The answer is France.',
  'First, The Statement (Film) is a movie from France. Second, A Self Made Hero is also a movie from France. The answer is France.'],
 'id': '9c4a7a4f090c11ebbdadac1f6bf848b6',
 'question': 'Are the movies The Statement (Film) and A Self Made Hero, from the same country?',
 'right_answer': 'yes'}

In [20]:
raw_pred

{'responses': ['First, the director of Broadway Or Bust was born in 1931. Second, the director of Ballad Of Tara was born in 1943. The answer is 1931.',
  'First, the director of Broadway Or Bust, Rick McKay, was born on January 20, 1962. Second, the director of Ballad Of Tara, Henry Jaglom, was born on January 26, 1938. The answer is 1938.',
  'First, the director of "Broadway Or Bust" was born in 1968. Second, the director of "Ballad Of Tara" was born in 1966. The answer is 1966.',
  'First, the director of "Broadway Or Bust" is Greg Vander Veer. Second, Greg Vander Veer was born on February 20, 1950. \nFirst, the director of "Ballad Of Tara" is Jorge Ameer. Second, Jorge Ameer was born on March 15, 1962. \nThe answer is 1950.',
  'First, "Broadway Or Bust" was directed by Robert N. Zagone. Second, Robert N. Zagone was born on November 17, 1960. \n\nFirst, "Ballad Of Tara" was directed by Chandra Siddhartha. Second, Chandra Siddhartha was born on October 9, 1968. \n\nThe answer is 19

In [22]:
print(cot_prompt)


Instructions:
1. You must always answer using the following chain-of-thought format.
2. Explicitly state the final answer in the form: "The answer is <final_answer>."
3. Ensure the final answer is a single word or entity (not a complete sentence).
4. Refer to the examples below to guide your responses.

Examples:

Q: This British racing driver came in third at the 2014 Bahrain GP2 Series round and was born in what year 
A: First, at the 2014 Bahrain GP2 Series round, DAMS driver Jolyon Palmer came in third. Second, Jolyon Palmer (born 20 January 1991) is a British racing driver. The answer is 1991. 

Q: What band did Antony King work with that formed in 1985 in Manchester? 
A: First, Antony King worked as house engineer for Simply Red. Second, Simply Red formed in 1985 in Manchester. The answer is Simply Red.  

Q: How many inhabitants were in the city close to where Alberta Ferretti’s studios was located? 
A: First, Alberta Ferretti’s studio is near Rimini. Second, Rimini is a city o

#### Mapping Data with Anomalies in the Original Run with the new run values

In [23]:
df1 = pd.read_csv('ve_predictions_cot_iter_0_100.csv')
df1.shape

(100, 20)

In [24]:
df2 = pd.read_csv('misc/ve_predictions_cot_iter_100_250.csv')
df2.shape

(149, 20)

In [25]:
df1_normal = df1.loc[df1.answer != 'NOT ENOUGH INFO']
df1_normal.shape

(85, 20)

In [26]:
df2_normal = df2.loc[df2.answer !='NOT ENOUGH INFO']
df2_normal.shape

(117, 20)

In [27]:
df_normal_all = pd.concat([df1_normal, df2_normal], axis =0)
df_normal_all.shape

(202, 20)

In [36]:
df1_abnormal_ids= list(df1.loc[df1.answer =='NOT ENOUGH INFO']['id'])

len(df1_abnormal_ids)

15

In [38]:
df2_abnormal_ids =list(df2.loc[df2.answer =='NOT ENOUGH INFO']['id'])
len(df2_abnormal_ids)

32

In [39]:
ids_abnormal = df1_abnormal_ids + df2_abnormal_ids
len(ids_abnormal)

47

### Mapping data from new run

In [41]:
df_new = pd.read_csv('misc/ve_predictions_cot_iter_latest_0_250.csv')
df_new.shape

(249, 21)

In [42]:
df_new_abnormal = df_new[df_new.id.isin(ids_abnormal)]
df_new_abnormal.shape

(47, 21)

In [44]:
list(df_new_abnormal.columns)

['Unnamed: 0',
 'response',
 'id',
 'question',
 'right_answer',
 'consistency',
 'rationale',
 'answer',
 'original_answers',
 'original_rationales',
 'consistency_time',
 'high_consistency',
 'verifying_questions',
 'time_verifying_answer',
 'time_context_and_verifying_answer',
 'context',
 'verifying_answers',
 'time_answer_again',
 'new_answer',
 'time_taken_sec',
 'match_flag']

In [45]:
df_new_abnormal.drop(columns='Unnamed: 0', inplace= True)

/var/folders/v5/0xf6m2_x0wx0j8k4xktnvxbr0000gn/T/ipykernel_8714/1846138354.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new_abnormal.drop(columns='Unnamed: 0', inplace= True)


In [46]:
df_new_abnormal.shape

(47, 20)

In [47]:
df_final = pd.concat([df_normal_all, df_new_abnormal], axis =0)
df_final.shape

(249, 20)

In [48]:
df_final.to_csv('misc/ve_predictions_cot_iter_final_0_250.csv')